In [1]:
import pandas as pd
import re
import os
from datetime import datetime
from IPython.display import display

# Cartella dove si trovano i file CSV
cartella = r"C:\Users\Antonio Maurizi\Desktop\PROGETTO ESAME EPICODE"

# Elenco dei file CSV che vogliamo unire
file_regionali = [
    "Reportistica_231231_Regioni.csv",
    "Reportistica_241231_Regioni.csv",
    "Reportistica_250930_Regioni.csv"
]

# Dizionario per rivedere e uniformare il nome di alcune colonne
# (perché nei vari file possono avere nomi leggermente diversi)
colonne_rettifica = {
    "Punti dichiarati": "Punti terminazione dichiarati",
    "Punti geograficamente distinti (passo 5m)": "Punti terminazione geograficamente distinti (passo 5m)",
    "Indirizzi postali distinti (se fornito/disponibile)": "Indirizzi postali distinti (se forniti/disponibili)",
    "Celle 20m x 20m raggiunte FTTH": "Celle 20m x 20m raggiunte FTTH",
    "Punti dichiarati FTTH": "Punti terminazione dichiarati FTTH",
    "Punti Geograficamente Distinti (passo 5m) FTTH": "Punti terminazione geograficamente distinti (passo 5m) FTTH",
    "Famiglie Residenti": "Famiglie Residenti (calcolato)",
    "Famiglie raggiunte FTTH": "Famiglie raggiunte FTTH (calcolato)",
}

# Funzione che si occupa di sistemare un file dopo la lettura
def pulisci_dataframe(df):
# Ripulisce i nomi delle colonne (toglie spazi e caratteri strani)
    df.columns = (
        df.columns
        .str.replace("\n", " ", regex=True)
        .str.replace(r"\\", " ", regex=True)
        .str.strip()
    )

# Controlla tutte le colonne di tipo testo
# e cerca di trasformare i numeri scritti "con il formato europeo"

    for col in df.columns:
        if df[col].dtype == "object":
# Rimuove eventuali punti (es: 1.000 → 1000)
# e converte la virgola in punto per i decimali (es: 10,5 → 10.5)

            df[col] = (
                df[col]
                .astype(str)
                .str.replace(".", "", regex=False)  # rimuove punti migliaia
                .str.replace(",", ".", regex=False)  # converte virgole in decimali
            )
            try:
                df[col] = pd.to_numeric(df[col])
            except Exception:
                pass

    return df

# Lista che conterrà tutti i file dopo la pulizia
dati_completi = []
for nome in file_regionali:
# Costruisce il percorso completo del file    
    percorso = os.path.join(cartella, nome)


# Legge il CSV (separatore ; e gestione caratteri speciali) e rimuove eventuali caratteri strani dalle intestazioni
    
    df = pd.read_csv(percorso, sep=";", encoding="latin-1", on_bad_lines="skip")
    df.columns = [c.replace("ï»¿", "").strip() for c in df.columns]

# Elimina righe completamente vuote
    df = df.dropna(how="all")

# Pulizia e rinomina e rinomina le colonne dove serve
    df = pulisci_dataframe(df)
    df = df.rename(columns=colonne_rettifica)

# Estrazione anno, mese, trimestre e data di riferimento dal nome file
    match = re.search(r"(\d{2})(\d{2})(\d{2})", nome)
    if match:
        anno = "20" + match.group(1)
        mese = match.group(2)
        giorno = match.group(3)
        trimestre = (
            "Q1" if mese == "03" else
            "Q2" if mese == "06" else
            "Q3" if mese == "09" else
            "Q4" if mese == "12" else None
        )
        df["Anno"] = int(anno)
        df["Trimestre"] = trimestre

#  Aggiungo anche la DataRiferimento (formato YYYY-MM-DD)
        df["DataRiferimento"] = datetime(int(anno), int(mese), int(giorno))

    print(f"Letto {nome} → {len(df)} righe, {len(df.columns)} colonne")
    dati_completi.append(df)

# Unione finale
Copertura_Regionale = pd.concat(dati_completi, ignore_index=True)

#  Visualizza risultato finale pulito
display(Copertura_Regionale.head(20))


Letto Reportistica_231231_Regioni.csv → 20 righe, 15 colonne
Letto Reportistica_241231_Regioni.csv → 20 righe, 15 colonne
Letto Reportistica_250930_Regioni.csv → 20 righe, 15 colonne


,Regione,Celle 20m x 20m raggiunte,Punti terminazione dichiarati,Punti terminazione geograficamente distinti (passo 5m),Indirizzi postali distinti (se forniti/disponibili),Celle 20m x 20m raggiunte FTTH,Punti terminazione dichiarati FTTH,Punti terminazione geograficamente distinti (passo 5m) FTTH,Indirizzi postali distinti (se forniti/disponibili) FTTH,Celle 20m x 20m raggiunte FTTC,Famiglie Residenti (calcolato),Famiglie raggiunte FTTH (calcolato),Anno,Trimestre,DataRiferimento
0,Abruzzo,297345,2282955,613632,1446106,120666,268215,224610,246185,193540,501425,288530,2023,Q4,2023-12-31
1,Basilicata,101340,1102771,283629,650937,29759,76411,67404,83155,72324,212350,91826,2023,Q4,2023-12-31
2,Calabria,367176,4780341,861845,2618856,56839,153522,111560,263546,329344,719265,259651,2023,Q4,2023-12-31
3,Campania,732837,6547433,1671979,4054498,276318,664236,543966,759860,537347,1975602,1424381,2023,Q4,2023-12-31
4,Emilia-Romagna,1073211,6230335,1899998,3390921,348646,697854,571505,493467,800319,1906953,1099272,2023,Q4,2023-12-31
5,Friuli Venezia Giulia,387957,2102047,618784,878805,129323,204644,176035,117829,324389,524917,299896,2023,Q4,2023-12-31
6,Lazio,881480,7284667,1895226,3479302,314721,698008,578465,706822,690133,2375856,1703292,2023,Q4,2023-12-31
7,Liguria,285175,2338769,569385,1143524,78924,259946,157922,246475,207478,709888,395168,2023,Q4,2023-12-31
8,Lombardia,1863332,10637234,3298240,4974597,596977,1041180,881526,597942,1393833,4214801,2467558,2023,Q4,2023-12-31
9,Marche,368726,2306815,709549,1537448,123774,214854,197188,154666,247364,590571,344707,2023,Q4,2023-12-31
